# MLOps Zoomcamp Week 1 - Notebook

Link to questions
https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2025/01-intro/homework.md

## 1. No of columns in DS

In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [3]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [4]:
df.shape[1] # shape = row,columns tuple

19

## 2. Computing duration and standard deviation

In [5]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [6]:
df['duration'].std()

np.float64(42.59435124195458)

## 3. Dropping outliers
% of dataset retained

In [7]:
before_length = df.shape[0]
print (before_length)
df = df[(df.duration >= 1) & (df.duration <= 60)]
after_length = df.shape[0]
print (after_length)

3066766
3009173


In [8]:
after_length / before_length * 100

98.1220282212598

## 4. One hot encoding

In [9]:
categorical = ['PULocationID', 'DOLocationID']
dv = DictVectorizer()
train_dicts = df[categorical].astype(str).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [10]:
X_train.shape

(3009173, 515)

## 5. Training The Model

In [11]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [12]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [13]:
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [14]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [16]:
ytrain_pred = lr.predict(X_train)
root_mean_squared_error(ytrain_pred, y_train)

7.649261946248764

## 6. Evaluating the model

In [17]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [18]:
y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

7.811817544074326